import pandas as pd

In [51]:
df = pd.read_csv('train_modified.csv') # Important!!
print(df.shape)

(903653, 56)


In [52]:
#Verify that campaign code is useless
column = df['trafficSource.campaignCode']
new_col = column.dropna()
print(new_col)

61493    11251kjhkvahf
Name: trafficSource.campaignCode, dtype: object


In [53]:
#Print all the column names
print(df.columns.values)

['Unnamed: 0' 'channelGrouping' 'date' 'fullVisitorId' 'sessionId'
 'socialEngagementType' 'visitId' 'visitNumber' 'visitStartTime'
 'device.browser' 'device.browserSize' 'device.browserVersion'
 'device.deviceCategory' 'device.flashVersion' 'device.isMobile'
 'device.language' 'device.mobileDeviceBranding' 'device.mobileDeviceInfo'
 'device.mobileDeviceMarketingName' 'device.mobileDeviceModel'
 'device.mobileInputSelector' 'device.operatingSystem'
 'device.operatingSystemVersion' 'device.screenColors'
 'device.screenResolution' 'geoNetwork.city' 'geoNetwork.cityId'
 'geoNetwork.continent' 'geoNetwork.country' 'geoNetwork.latitude'
 'geoNetwork.longitude' 'geoNetwork.metro' 'geoNetwork.networkDomain'
 'geoNetwork.networkLocation' 'geoNetwork.region'
 'geoNetwork.subContinent' 'totals.bounces' 'totals.hits'
 'totals.newVisits' 'totals.pageviews' 'totals.transactionRevenue'
 'totals.visits' 'trafficSource.adContent'
 'trafficSource.adwordsClickInfo.adNetworkType'
 'trafficSource.adwordsC

In [77]:
#View some non-zero transactions
transactionCol = df['totals.transactionRevenue']
nonZeroTransactions = transactionCol.dropna()
print(nonZeroTransactions)

752        37860000.0
753       306670000.0
799        68030000.0
802        26250000.0
859       574150000.0
866         8380000.0
893       395730000.0
910        24080000.0
922        35480000.0
925        35080000.0
930        81500000.0
942       546320000.0
971        63840000.0
974       117340000.0
976        43710000.0
999        34480000.0
1000       35390000.0
1008      339030000.0
1020       33670000.0
1022      635500000.0
1026      103240000.0
1027      305470000.0
1037        5150000.0
1041       90850000.0
1054       10590000.0
1055       25680000.0
1056      338970000.0
2774      593910000.0
3161       44790000.0
3268       30390000.0
             ...     
899951     43430000.0
903089     44790000.0
903118    175800000.0
903122     87990000.0
903167     14370000.0
903168    155970000.0
903189     75180000.0
903194    350000000.0
903201    135920000.0
903211    159980000.0
903222     23180000.0
903228     22970000.0
903234    463600000.0
903235     79990000.0
903243    

In [108]:
#Print some basic transaction stats
print('No of non-zero transactions : ', nonZeroTransactions.size)

transactionsSum = nonZeroTransactions.sum()
print('Total revenue is : ', transactionsSum)

print('Max revenue is : ', nonZeroTransactions.max())
print('Min revenue is : ', nonZeroTransactions.min())
print('Average revenue per transaction is : ', nonZeroTransactions.mean())

No of non-zero transactions :  11515
Total revenue is :  1540071240000.0
Max revenue is :  23129500000.0
Min revenue is :  10000.0
Average revenue per transaction is :  133744788.53669128


In [120]:
#Print some basic stats about unique users
uniqueUsersCount = df['fullVisitorId'].unique().size
print('No of unique users : ', uniqueUsersCount)

usersTransactionsSum = df.groupby('fullVisitorId')['totals.transactionRevenue'].sum()
payingUsers = usersTransactionsSum[usersTransactionsSum!=0]
payingUsersCount = payingUsers.size

print('No of paying users : ', payingUsersCount)

aveRevenuePerpayingUser = usersTransactionsSum.sum() / payingUsersCount

print('Ave. revenue per paying user : ', aveRevenuePerpayingUser)

#Some other comparisons about paying users
print('Percentage of users that pay : ', payingUsersCount/uniqueUsersCount)

No of unique users :  723803
No of paying users :  10114
Ave. revenue per paying user :  152271231.95570496
Percentage of users that pay :  0.013973415418283704
